In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from clean_data import load_clean_data as lcd
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [2]:
a = np.arange(100).reshape(10, 10)
a

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])

In [3]:
b = np.arange(20, 30)
b

array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [4]:
sess = tf.Session()

In [5]:
x = tf.constant(a)
y = tf.constant(b)
xy = tf.mul(x, y)

In [6]:
sess.run(xy)

array([[   0,   21,   44,   69,   96,  125,  156,  189,  224,  261],
       [ 200,  231,  264,  299,  336,  375,  416,  459,  504,  551],
       [ 400,  441,  484,  529,  576,  625,  676,  729,  784,  841],
       [ 600,  651,  704,  759,  816,  875,  936,  999, 1064, 1131],
       [ 800,  861,  924,  989, 1056, 1125, 1196, 1269, 1344, 1421],
       [1000, 1071, 1144, 1219, 1296, 1375, 1456, 1539, 1624, 1711],
       [1200, 1281, 1364, 1449, 1536, 1625, 1716, 1809, 1904, 2001],
       [1400, 1491, 1584, 1679, 1776, 1875, 1976, 2079, 2184, 2291],
       [1600, 1701, 1804, 1909, 2016, 2125, 2236, 2349, 2464, 2581],
       [1800, 1911, 2024, 2139, 2256, 2375, 2496, 2619, 2744, 2871]])

In [10]:
churn = pd.read_csv('data/churn.csv')

In [11]:
churn.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4


In [12]:
df, X, y = lcd()

In [13]:
X.shape

(50000, 13)

In [14]:
y.shape

(50000,)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [16]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
model.coef_

array([[  3.32820408e-02,   1.57197882e-01,   1.35770184e-01,
          2.24401251e-01,  -4.80727403e-03,  -9.55086610e-02,
         -8.34610987e-01,  -1.36584803e-04,  -3.71074749e-02,
          5.48977928e-03,  -1.03534154e+00,  -1.62733126e+00,
         -4.73287434e-01]])

In [18]:
model.intercept_

array([ 0.61260535])

In [19]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.67      0.47      0.55      4564
          1       0.74      0.87      0.80      7936

avg / total       0.71      0.72      0.71     12500



In [20]:
logistic_regression_acc = accuracy_score(y_test, y_pred)
print(logistic_regression_acc)

0.72184


In [21]:
model.coef_.reshape(-1, 1).shape

(13, 1)

In [22]:
coefs = tf.constant(model.coef_.reshape(-1, 1), dtype=tf.float32) # important! dtype must match x (what it's multiplied by)
intercept = tf.constant(model.intercept_, dtype=tf.float32)

In [23]:
x = tf.placeholder(tf.float32, [None, 13]) # 13 features in input data
y = tf.placeholder(tf.float32, [None, 1]) # either a churn (1) or not (0)

In [24]:
pred = tf.nn.sigmoid(tf.matmul(x, coefs) + intercept) # sigmoid makes it logistic

In [25]:
sess = tf.Session()

In [26]:
init = tf.initialize_all_variables()
sess.run(init)

In [27]:
y_pred = sess.run(pred, feed_dict={x: X_test})

In [28]:
y_pred.sum()

7939.373

In [29]:
y_pred.shape

(12500, 1)

In [30]:
y_pred_round = np.round(y_pred)

In [31]:
y_pred_round.sum()

9283.0

In [32]:
neural_net_acc = accuracy_score(y_test, y_pred_round)
print(neural_net_acc)
print(logistic_regression_acc)

0.72184
0.72184


Nice! Now let's define a cost function and fit it, to make sure the weights don't change.

In [243]:
sess = tf.Session()
#model.coef_.reshape(-1, 1)
#tf.zeros([13, 1]
#model.intercept_
#tf.zeros([1])

coefs = tf.Variable(model.coef_.reshape(-1, 1), dtype=tf.float32) # important! dtype must match x (what it's multiplied by)
intercept = tf.Variable(model.intercept_, dtype=tf.float32)

In [244]:
x = tf.placeholder(tf.float32, [None, 13]) # 13 features in input data
y = tf.placeholder(tf.float32, [None, 1]) # either a churn (1) or not (0)

In [245]:
pred = tf.nn.sigmoid(tf.matmul(x, coefs) + intercept) # sigmoid makes it logistic

In [246]:
cost = tf.reduce_mean(tf.reduce_sum(tf.squared_difference(pred, y), reduction_indices=1))/2.

In [247]:
# Parameters
learning_rate = 0.001

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [594]:
tf.trainable_variables()

In [249]:
init = tf.initialize_all_variables()
sess.run(init)

In [250]:
coefs.eval(session=sess)

array([[  3.32820415e-02],
       [  1.57197878e-01],
       [  1.35770187e-01],
       [  2.24401250e-01],
       [ -4.80727386e-03],
       [ -9.55086574e-02],
       [ -8.34610999e-01],
       [ -1.36584800e-04],
       [ -3.71074751e-02],
       [  5.48977917e-03],
       [ -1.03534150e+00],
       [ -1.62733126e+00],
       [ -4.73287433e-01]], dtype=float32)

In [251]:
model.coef_.reshape(-1, 1)

array([[  3.32820408e-02],
       [  1.57197882e-01],
       [  1.35770184e-01],
       [  2.24401251e-01],
       [ -4.80727403e-03],
       [ -9.55086610e-02],
       [ -8.34610987e-01],
       [ -1.36584803e-04],
       [ -3.71074749e-02],
       [  5.48977928e-03],
       [ -1.03534154e+00],
       [ -1.62733126e+00],
       [ -4.73287434e-01]])

In [252]:
cs = sess.run(cost, feed_dict=({x: X_train, y: y_train.reshape(-1, 1)}))

In [253]:
cs

0.094842069

In [254]:
sess.run([optimizer, cost], feed_dict=({x: X_train, y: y_train.reshape(-1, 1)}))

[None, 0.094842069]

In [255]:
cs = sess.run(cost, feed_dict=({x: X_train, y: y_train.reshape(-1, 1)}))
print(cs)

0.0948405


In [268]:
for i in range(100):
    sess.run([optimizer, cost], feed_dict=({x: X_train, y: y_train.reshape(-1, 1)}))
    # y needs to be a row vector

In [284]:
y_pred = sess.run(pred, feed_dict=({x: X_test}))

y_pred_round = np.round(y_pred)

print(accuracy_score(y_test, y_pred_round))

FailedPreconditionError: Attempting to use uninitialized value Variable_24
	 [[Node: Variable_24/read = Identity[T=DT_FLOAT, _class=["loc:@Variable_24"], _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_24)]]
Caused by op 'Variable_24/read', defined at:
  File "/home/nate/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/nate/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/nate/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/nate/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/home/nate/anaconda3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/nate/anaconda3/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQ

Sweet! We can actually get a slightly better score than with just plain logistic regression.  Make sure to keep the learning rate low since we're close to the answer.  When I tried with the learning rate as 0.01 it actually made the fit worse.  Now let's make a NN with one hidden layer.

In [272]:
deep_sess = tf.Session()

coefs = tf.Variable(model.coef_.reshape(-1, 1), dtype=tf.float32) # important! dtype must match x (what it's multiplied by)
intercept = tf.Variable(model.intercept_, dtype=tf.float32)

In [ ]:
x = tf.placeholder(tf.float32, [None, 13]) # 13 features in input data
y = tf.placeholder(tf.float32, [None, 1]) # either a churn (1) or not (0)

In [277]:
n_input = 13
n_hidden_1 = 13*13
n_out = 1
weights_1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
biases_1 = tf.Variable(tf.random_normal([n_hidden_1]))
layer_1 = tf.add(tf.matmul(x, weights_1), biases_1)
layer_1 = tf.nn.relu(layer_1)

weights_out = tf.Variable(tf.random_normal([n_hidden_1, n_out]))
biases_out = tf.Variable(tf.random_normal([n_out]))
out_layer = tf.matmul(layer_1, weights_out) + biases_out

In [279]:
pred = tf.nn.sigmoid(out_layer) # sigmoid makes it logistic

cost = tf.reduce_mean(tf.reduce_sum(tf.squared_difference(pred, y), reduction_indices=1))/2.

# Parameters
learning_rate = 0.01

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [281]:
init = tf.initialize_all_variables()
deep_sess.run(init)

In [328]:
for i in range(100):
    deep_sess.run([optimizer, cost], feed_dict=({x: X_train, y: y_train.reshape(-1, 1)}))

In [329]:
y_pred = deep_sess.run(pred, feed_dict=({x: X_test}))

y_pred_round = np.round(y_pred)

print(accuracy_score(y_test, y_pred_round))

0.49504


Seems to max out at 49.5% accuracy...

Started with learning rate at 10, then decreased to 1, 0.1, 0.01.  Big jump from mid-30s accuracy to 49% all of a sudden with learning rate=10.

Try some different number of hidden nodes...

In [330]:
deep_sess2 = tf.Session()

coefs = tf.Variable(model.coef_.reshape(-1, 1), dtype=tf.float32) # important! dtype must match x (what it's multiplied by)
intercept = tf.Variable(model.intercept_, dtype=tf.float32)

In [331]:
x = tf.placeholder(tf.float32, [None, 13]) # 13 features in input data
y = tf.placeholder(tf.float32, [None, 1]) # either a churn (1) or not (0)

In [332]:
n_input = 13
n_hidden_1 = 13
n_out = 1
weights_1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
biases_1 = tf.Variable(tf.random_normal([n_hidden_1]))
layer_1 = tf.add(tf.matmul(x, weights_1), biases_1)
layer_1 = tf.nn.relu(layer_1)

weights_out = tf.Variable(tf.random_normal([n_hidden_1, n_out]))
biases_out = tf.Variable(tf.random_normal([n_out]))
out_layer = tf.matmul(layer_1, weights_out) + biases_out

In [348]:
pred = tf.nn.sigmoid(out_layer) # sigmoid makes it logistic

cost = tf.reduce_mean(tf.reduce_sum(tf.squared_difference(pred, y), reduction_indices=1))/2.

# Parameters
learning_rate = 0.01

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [335]:
init = tf.initialize_all_variables()
deep_sess2.run(init)

In [349]:
for i in range(100):
    deep_sess2.run([optimizer, cost], feed_dict=({x: X_train, y: y_train.reshape(-1, 1)}))

In [350]:
y_pred = deep_sess2.run(pred, feed_dict=({x: X_test}))

y_pred_round = np.round(y_pred)

print(accuracy_score(y_test, y_pred_round))

0.63488


Seems to max out around 63.5% accuracy after 100 iteration with learning rate = 10.

In [351]:
deep_sess3 = tf.Session()

coefs = tf.Variable(model.coef_.reshape(-1, 1), dtype=tf.float32) # important! dtype must match x (what it's multiplied by)
intercept = tf.Variable(model.intercept_, dtype=tf.float32)

In [352]:
x = tf.placeholder(tf.float32, [None, 13]) # 13 features in input data
y = tf.placeholder(tf.float32, [None, 1]) # either a churn (1) or not (0)

In [353]:
n_input = 13
n_hidden_1 = 6
n_out = 1
weights_1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
biases_1 = tf.Variable(tf.random_normal([n_hidden_1]))
layer_1 = tf.add(tf.matmul(x, weights_1), biases_1)
layer_1 = tf.nn.relu(layer_1)

weights_out = tf.Variable(tf.random_normal([n_hidden_1, n_out]))
biases_out = tf.Variable(tf.random_normal([n_out]))
out_layer = tf.matmul(layer_1, weights_out) + biases_out

In [361]:
pred = tf.nn.sigmoid(out_layer) # sigmoid makes it logistic

cost = tf.reduce_mean(tf.reduce_sum(tf.squared_difference(pred, y), reduction_indices=1))/2.

# Parameters
learning_rate = 0.1

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [362]:
init = tf.initialize_all_variables()
deep_sess3.run(init)

In [371]:
for i in range(100):
    deep_sess3.run([optimizer, cost], feed_dict=({x: X_train, y: y_train.reshape(-1, 1)}))

In [372]:
y_pred = deep_sess3.run(pred, feed_dict=({x: X_test}))

y_pred_round = np.round(y_pred)

print(accuracy_score(y_test, y_pred_round))

0.63472


Seems like accuracy went down slightly.  Maybe if we try something between 13 and 13*13 for our number of nodes...

In [373]:
deep_sess4 = tf.Session()

coefs = tf.Variable(model.coef_.reshape(-1, 1), dtype=tf.float32) # important! dtype must match x (what it's multiplied by)
intercept = tf.Variable(model.intercept_, dtype=tf.float32)

In [374]:
x = tf.placeholder(tf.float32, [None, 13]) # 13 features in input data
y = tf.placeholder(tf.float32, [None, 1]) # either a churn (1) or not (0)

In [375]:
n_input = 13
n_hidden_1 = 13*3
n_out = 1
weights_1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
biases_1 = tf.Variable(tf.random_normal([n_hidden_1]))
layer_1 = tf.add(tf.matmul(x, weights_1), biases_1)
layer_1 = tf.nn.relu(layer_1)

weights_out = tf.Variable(tf.random_normal([n_hidden_1, n_out]))
biases_out = tf.Variable(tf.random_normal([n_out]))
out_layer = tf.matmul(layer_1, weights_out) + biases_out

In [411]:
pred = tf.nn.sigmoid(out_layer) # sigmoid makes it logistic

cost = tf.reduce_mean(tf.reduce_sum(tf.squared_difference(pred, y), reduction_indices=1))/2.

# Parameters
learning_rate = 0.01

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [377]:
init = tf.initialize_all_variables()
deep_sess4.run(init)

In [412]:
for i in range(100):
    deep_sess4.run([optimizer, cost], feed_dict=({x: X_train, y: y_train.reshape(-1, 1)}))

In [413]:
y_pred = deep_sess4.run(pred, feed_dict=({x: X_test}))

y_pred_round = np.round(y_pred)

print(accuracy_score(y_test, y_pred_round))

0.47776


Seems like this achitecture isn't going to work, it maxs out at 47.776% acc. Let's try again, but this time with the logistic part first (scaling each component, essentially), then the deep net after that.

In [414]:
deep_sess5 = tf.Session()

coefs = tf.Variable(model.coef_.reshape(-1, 1), dtype=tf.float32) # important! dtype must match x (what it's multiplied by)
intercept = tf.Variable(model.intercept_, dtype=tf.float32)

In [415]:
x = tf.placeholder(tf.float32, [None, 13]) # 13 features in input data
y = tf.placeholder(tf.float32, [None, 1]) # either a churn (1) or not (0)

In [449]:
n_input = 13
n_hidden_1 = 13*2
n_out = 1

keep_prob = tf.placeholder(tf.float32)

weights_1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
biases_1 = tf.Variable(tf.random_normal([n_hidden_1]))
layer_1 = tf.add(tf.matmul(x, weights_1), biases_1)
layer_1 = tf.nn.relu(layer_1)

layer_1 = tf.nn.dropout(layer_1, keep_prob)

weights_out = tf.Variable(tf.random_normal([n_hidden_1, n_out]))
biases_out = tf.Variable(tf.random_normal([n_out]))
out_layer = tf.matmul(layer_1, weights_out) + biases_out

In [450]:
pred = tf.nn.sigmoid(out_layer) # sigmoid makes it logistic

cost = tf.reduce_mean(tf.reduce_sum(tf.squared_difference(pred, y), reduction_indices=1))/2.

# Parameters
learning_rate = 0.1

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [451]:
init = tf.initialize_all_variables()
deep_sess5.run(init)

In [478]:
keep_prob = 0.5
for i in range(100):
    deep_sess5.run([optimizer, cost], feed_dict=({x: X_train, y: y_train.reshape(-1, 1)}))

In [479]:
keep_prob = 1
y_pred = deep_sess5.run(pred, feed_dict=({x: X_test}))

y_pred_round = np.round(y_pred)

print(accuracy_score(y_test, y_pred_round))

0.6276


Maybe dropout helps?  Let's compare with a model we tried without dropout.

In [484]:
deep_sess6 = tf.Session()

coefs = tf.Variable(model.coef_.reshape(-1, 1), dtype=tf.float32) # important! dtype must match x (what it's multiplied by)
intercept = tf.Variable(model.intercept_, dtype=tf.float32)

In [485]:
x = tf.placeholder(tf.float32, [None, 13]) # 13 features in input data
y = tf.placeholder(tf.float32, [None, 1]) # either a churn (1) or not (0)

In [498]:
n_input = 13
n_hidden_1 = 6
n_out = 1

keep_prob = tf.placeholder(tf.float32)

weights_1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
biases_1 = tf.Variable(tf.random_normal([n_hidden_1]))
layer_1 = tf.add(tf.matmul(x, weights_1), biases_1)
layer_1 = tf.nn.relu(layer_1)

layer_1 = tf.nn.dropout(layer_1, keep_prob)

weights_out = tf.Variable(tf.random_normal([n_hidden_1, n_out]))
biases_out = tf.Variable(tf.random_normal([n_out]))
out_layer = tf.matmul(layer_1, weights_out) + biases_out

In [557]:
pred = tf.nn.sigmoid(out_layer) # sigmoid makes it logistic

cost = tf.reduce_mean(tf.reduce_sum(tf.squared_difference(pred, y), reduction_indices=1))/2.

# Parameters
learning_rate = 1

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [520]:
init = tf.initialize_all_variables()
deep_sess6.run(init)

In [558]:
for i in range(100):
    deep_sess6.run([optimizer, cost], feed_dict=({x: X_train, y: y_train.reshape(-1, 1), keep_prob: 0.5}))

In [559]:
y_pred = deep_sess6.run(pred, feed_dict=({x: X_test, keep_prob: 1.0}))

y_pred_round = np.round(y_pred)

print(accuracy_score(y_test, y_pred_round))

0.6348


Before we got 63.472...not a huge improvement.

Alright.  Need at least 2 hidden layers for full generality.

In [560]:
deep_sess7 = tf.Session()

coefs = tf.Variable(model.coef_.reshape(-1, 1), dtype=tf.float32) # important! dtype must match x (what it's multiplied by)
intercept = tf.Variable(model.intercept_, dtype=tf.float32)

In [561]:
x = tf.placeholder(tf.float32, [None, 13]) # 13 features in input data
y = tf.placeholder(tf.float32, [None, 1]) # either a churn (1) or not (0)

In [562]:
n_input = 13
n_hidden_1 = 13*2
n_hidden_2 = 13*2
n_out = 1

keep_prob = tf.placeholder(tf.float32)

weights_1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
biases_1 = tf.Variable(tf.random_normal([n_hidden_1]))
layer_1 = tf.add(tf.matmul(x, weights_1), biases_1)
layer_1 = tf.nn.relu(layer_1)
layer_1 = tf.nn.dropout(layer_1, keep_prob)

weights_2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
biases_2 = tf.Variable(tf.random_normal([n_hidden_2]))
layer_2 = tf.add(tf.matmul(layer_1, weights_2), biases_2)
layer_2 = tf.nn.relu(layer_2)
layer_2 = tf.nn.dropout(layer_2, keep_prob)

weights_out = tf.Variable(tf.random_normal([n_hidden_2, n_out]))
biases_out = tf.Variable(tf.random_normal([n_out]))
out_layer = tf.matmul(layer_2, weights_out) + biases_out

In [571]:
pred = tf.nn.sigmoid(out_layer) # sigmoid makes it logistic

cost = tf.reduce_mean(tf.reduce_sum(tf.squared_difference(pred, y), reduction_indices=1))/2.

# Parameters
learning_rate = 10

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [564]:
init = tf.initialize_all_variables()
deep_sess7.run(init)

In [574]:
for i in range(100):
    deep_sess7.run([optimizer, cost], feed_dict=({x: X_train, y: y_train.reshape(-1, 1), keep_prob: 0.5}))

In [575]:
y_pred = deep_sess7.run(pred, feed_dict=({x: X_test, keep_prob: 1.0}))

y_pred_round = np.round(y_pred)

print(accuracy_score(y_test, y_pred_round))

0.63488


Seemed to converge much faster, but not much better than a single layer.  Let's keep adding layers I guess...

In [595]:
tf.reset_default_graph()

In [596]:
deep_sess8 = tf.Session()

In [597]:
x = tf.placeholder(tf.float32, [None, 13]) # 13 features in input data
y = tf.placeholder(tf.float32, [None, 1]) # either a churn (1) or not (0)

In [598]:
n_input = 13
n_hidden_1 = 13*2
n_hidden_2 = 13*2
n_hidden_3 = 13*2
n_out = 1

keep_prob = tf.placeholder(tf.float32)

weights_1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
biases_1 = tf.Variable(tf.random_normal([n_hidden_1]))
layer_1 = tf.add(tf.matmul(x, weights_1), biases_1)
layer_1 = tf.nn.relu(layer_1)
layer_1 = tf.nn.dropout(layer_1, keep_prob)

weights_2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
biases_2 = tf.Variable(tf.random_normal([n_hidden_2]))
layer_2 = tf.add(tf.matmul(layer_1, weights_2), biases_2)
layer_2 = tf.nn.relu(layer_2)
layer_2 = tf.nn.dropout(layer_2, keep_prob)

weights_3 = tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3]))
biases_3 = tf.Variable(tf.random_normal([n_hidden_3]))
layer_3 = tf.add(tf.matmul(layer_2, weights_3), biases_3)
layer_3 = tf.nn.relu(layer_3)
layer_3 = tf.nn.dropout(layer_3, keep_prob)

weights_out = tf.Variable(tf.random_normal([n_hidden_3, n_out]))
biases_out = tf.Variable(tf.random_normal([n_out]))
out_layer = tf.matmul(layer_3, weights_out) + biases_out

In [610]:
pred = tf.nn.sigmoid(out_layer) # sigmoid makes it logistic

cost = tf.reduce_mean(tf.reduce_sum(tf.squared_difference(pred, y), reduction_indices=1))/2.

# Parameters
learning_rate = 1

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [600]:
init = tf.initialize_all_variables()
deep_sess8.run(init)

In [613]:
for i in range(100):
    deep_sess8.run([optimizer, cost], feed_dict=({x: X_train, y: y_train.reshape(-1, 1), keep_prob: 0.5}))

In [614]:
y_pred = deep_sess8.run(pred, feed_dict=({x: X_test, keep_prob: 1.0}))

y_pred_round = np.round(y_pred)

print(accuracy_score(y_test, y_pred_round))

0.63488


In [603]:
tf.trainable_variables()

In [673]:
tf.reset_default_graph()

In [674]:
deep_sess8 = tf.Session()

In [675]:
x = tf.placeholder(tf.float32, [None, 13]) # 13 features in input data
y = tf.placeholder(tf.float32, [None, 1]) # either a churn (1) or not (0)

In [676]:
n_input = 13
n_hidden_1 = 13*10
n_hidden_2 = 13*5
n_hidden_3 = 13
n_out = 1

keep_prob = tf.placeholder(tf.float32)

weights_1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
biases_1 = tf.Variable(tf.random_normal([n_hidden_1]))
layer_1 = tf.add(tf.matmul(x, weights_1), biases_1)
layer_1 = tf.nn.relu(layer_1)
layer_1 = tf.nn.dropout(layer_1, keep_prob)

weights_2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
biases_2 = tf.Variable(tf.random_normal([n_hidden_2]))
layer_2 = tf.add(tf.matmul(layer_1, weights_2), biases_2)
layer_2 = tf.nn.relu(layer_2)
layer_2 = tf.nn.dropout(layer_2, keep_prob)

weights_3 = tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3]))
biases_3 = tf.Variable(tf.random_normal([n_hidden_3]))
layer_3 = tf.add(tf.matmul(layer_2, weights_3), biases_3)
layer_3 = tf.nn.relu(layer_3)
layer_3 = tf.nn.dropout(layer_3, keep_prob)

weights_out = tf.Variable(tf.random_normal([n_hidden_3, n_out]))
biases_out = tf.Variable(tf.random_normal([n_out]))
out_layer = tf.matmul(layer_3, weights_out) + biases_out

In [677]:
pred = tf.nn.sigmoid(out_layer) # sigmoid makes it logistic

cost = tf.reduce_mean(tf.reduce_sum(tf.squared_difference(pred, y), reduction_indices=1))/2.

# Parameters
learning_rate = 1

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [ ]:
init = tf.initialize_all_variables()
deep_sess8.run(init)

In [ ]:
for i in range(100):
    deep_sess8.run([optimizer, cost], feed_dict=({x: X_train, y: y_train.reshape(-1, 1), keep_prob: 0.5}))

In [ ]:
y_pred = deep_sess8.run(pred, feed_dict=({x: X_test, keep_prob: 1.0}))

y_pred_round = np.round(y_pred)

print(accuracy_score(y_test, y_pred_round))